<a href="https://colab.research.google.com/github/Lnguyen19/Avatar_generator/blob/main/character_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()



Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"lam8134","key":"154eb2b43b31cd43cec727b1bf231669"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d aadilmalik94/animecharacterfaces

 98% 747M/760M [00:06<00:00, 107MB/s]
100% 760M/760M [00:06<00:00, 124MB/s]


In [ ]:
!unzip animecharacterfaces.zip

Streaming output truncated to the last 5000 lines.
  inflating: animeface-character-dataset/data/face_323_149_22.png  
  inflating: animeface-character-dataset/data/face_323_149_25.png  
  inflating: animeface-character-dataset/data/face_323_149_32.png  
  inflating: animeface-character-dataset/data/face_323_149_38.png  
  inflating: animeface-character-dataset/data/face_323_149_56.png  
  inflating: animeface-character-dataset/data/face_323_149_7.png  
  inflating: animeface-character-dataset/data/face_323_149_81.png  
  inflating: animeface-character-dataset/data/face_323_174_148.png  
  inflating: animeface-character-dataset/data/face_323_174_151.png  
  inflating: animeface-character-dataset/data/face_323_174_154.png  
  inflating: animeface-character-dataset/data/face_323_174_44.png  
  inflating: animeface-character-dataset/data/face_323_174_52.png  
  inflating: animeface-character-dataset/data/face_323_174_54.png  
  inflating: animeface-character-dataset/data/face_323_174_92.p

In [ ]:
import numpy as np
import os

# for deep learning:
import keras
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Dropout, ZeroPadding2D
from keras.layers import BatchNormalization, Flatten
from keras.models import Sequential, Model, load_model
from keras.layers import Activation
from keras.layers import Reshape # new!
from keras.layers import Conv2DTranspose, UpSampling2D # new!
from keras.optimizers import RMSprop # new!
from keras.layers import LeakyReLU
from keras.optimizers import Adam
from keras.optimizers import RMSprop
# for plotting:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
from PIL import Image

In [ ]:
# prompt: read all the images in the directory "/content/animeface-character-dataset/animeface-character-dataset/data", the images are png


from PIL import Image
training_data = []
IMAGE_SIZE = 128
IMAGE_CHANNELS = 3
#def read_images(directory,size):
 # images = []
 # for filename in os.listdir(directory):
 #   if filename.endswith(".png"):
 #     image = Image.open(os.path.join(directory, filename))
 #     image = image.resize(size)
 #     image = np.array(image)
 #     images.append(image)
 # return images
images_path = '/content/animeface-character-dataset/animeface-character-dataset/data'
images_list = []

In [ ]:

411
#training_data= read_images('/content/animeface-character-dataset/animeface-character-dataset/data',IMAGE_SIZE)
print('resizing...')
for filename in os.listdir(images_path):
    path = os.path.join(images_path, filename)
    try:
        image = Image.open(path).resize((IMAGE_SIZE, IMAGE_SIZE), Image.LANCZOS)
    except OSError:
        continue
    images_list.append(np.array(image))

# Now, convert your list to a numpy array and assign it to training_data
training_data = np.array(images_list)
training_data = training_data / 127.5 - 1

print('saving files')
np.save('faces_data.npy',training_data)




resizing...
saving files


In [ ]:
PREVIEW_ROWS = 1
PREVIEW_COLS = 1
PREVIEW_MARGIN = 4
SAVE_FREQ = 100

#SIZE VECTOR TO GENERATE IMAGES FROM
NOISE_SIZE= 100

#configuration
epochs = 1200
batch_size = 64

generate_res = 3
IMAGE_SIZE = 128
IMAGE_CHANNELS = 3

training_data = np.load(os.path.join('/content/', 'faces_data.npy'))

In [ ]:
def build_discriminator(image_shape):
  model = Sequential()
  a = 0.5
  m = 0.85
  d=0.8
  model.add(Conv2D(32,kernel_size = 3, strides = 2, input_shape = image_shape, padding = "same"))
  model.add(LeakyReLU(alpha = a))
  model.add(Dropout(d))

  model.add(Conv2D(64,kernel_size = 3,strides = 2,padding = "same"))
  model.add(ZeroPadding2D(padding = ((0,1),(0,1))))
  model.add(BatchNormalization(momentum= m))
  model.add(LeakyReLU(alpha = a))
  model.add(Dropout(d))

  model.add(Conv2D(128,kernel_size = 3,strides = 2,padding = "same"))
  model.add(BatchNormalization(momentum=m))
  model.add(LeakyReLU(alpha = a))
  model.add(Dropout(d))

  #model.add(Conv2D(256,kernel_size = 3,strides = 1,padding = "same"))
  #model.add(BatchNormalization(momentum=m))
  #model.add(LeakyReLU(alpha = a))
  #model.add(Dropout(d))

  #model.add(Conv2D(256*2,kernel_size = 3,strides = 1,padding = "same"))
  #model.add(BatchNormalization(momentum=m))
  #model.add(LeakyReLU(alpha = a))
  #model.add(Dropout(d))

  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))

  input_image = Input(shape=image_shape)
  validity = model(input_image)
  return Model(input_image, validity)




In [ ]:
def build_generator(noise_size, channels):
  m = 0.6
  a = 0.2
  kernelSize = 3
  model = Sequential()
  model.add(Dense(4*4*256,activation = 'relu', input_dim = noise_size))
  model.add(Reshape((4, 4, 256)))

  model.add(UpSampling2D())
  model.add(Conv2D(256,kernel_size = kernelSize,padding = "same"))
  model.add(BatchNormalization(momentum = m))
  model.add(Activation("relu"))

  model.add(UpSampling2D())
  model.add(Conv2D(256*4,kernel_size = kernelSize, padding = "same"))
  model.add(BatchNormalization(momentum = m))
  model.add(Activation("relu"))

  for i in range(generate_res):
    model.add(UpSampling2D())
    model.add(Conv2D(256,kernel_size = kernelSize, padding = "same"))
    model.add(BatchNormalization(momentum =m))
    model.add(Activation("relu"))
    model.summary()

  model.add(Conv2D(channels,kernel_size = kernelSize, padding = "same"))
  model.add(Activation("tanh"))
  input = Input(shape= (noise_size,))
  generated_image = model(input)
  return Model(input,generated_image)

In [ ]:
import os
import numpy as np
from PIL import Image

def save_images(cnt,noise):
    image_array = np.full((
        PREVIEW_MARGIN + (PREVIEW_ROWS * (IMAGE_SIZE + PREVIEW_MARGIN)),
        PREVIEW_MARGIN + (PREVIEW_COLS * (IMAGE_SIZE + PREVIEW_MARGIN)), 3),
        255, dtype=np.uint8)

    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5
    image_count = 0

    for row in range(PREVIEW_ROWS):
        for col in range(PREVIEW_COLS):
            r = row * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
            c = col * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
            image_array[r:r + IMAGE_SIZE, c:c + IMAGE_SIZE] = generated_images[image_count] * 255
            image_count += 1

    output_path = '/content/output'
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    filename = os.path.join(output_path, f"trained-{cnt}.png")
    im = Image.fromarray(image_array.astype(np.uint8))
    im.save(filename)
    print(f"Saved {filename}")



In [ ]:
image_shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS)

discriminator_optimizer = Adam(1.5e-4, 0.5,clipvalue=0.5)
combined_optimizer = Adam(2e-4, 0.5)
# dis  = 1.5e-4
# combined = 2e-4
#optimizer = GradientAccumulation('adam', accumulation_steps=8)
#optimizer = AdamAccumulated(accumulation_steps=8)

discriminator = build_discriminator(image_shape)
discriminator.compile(loss="binary_crossentropy",optimizer=discriminator_optimizer, metrics=["accuracy"])
#discriminator.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.0008, decay=6e-8, clipvalue=1.0), metrics=['accuracy'])
generator = build_generator(NOISE_SIZE, IMAGE_CHANNELS)

random_input = Input(shape=(NOISE_SIZE,))

generated_image = generator(random_input)

discriminator.trainable = False

validity = discriminator(generated_image)

combined = Model(random_input, validity)
combined.compile(loss="binary_crossentropy",optimizer=combined_optimizer, metrics=["accuracy"])

y_real = np.ones((batch_size, 1))
y_fake = np.zeros((batch_size, 1))

#fixed_noise = tf.random.normal([PREVIEW_ROWS * PREVIEW_COLS, NOISE_SIZE], -1.0, 1.0, tf.float32)
fixed_noise = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, NOISE_SIZE)) # Possible fix #2: change from np.random.normal to tensorflow's tf.random.normal

cnt = 1
for epoch in range(epochs):
 idx = np.random.randint(0, training_data.shape[0], batch_size)
 x_real = training_data[idx]

 #noise = tf.random.normal([batch_size, NOISE_SIZE], -1.0, 1.0, tf.float32)
 noise= np.random.normal(0, 1, (batch_size, NOISE_SIZE)) # Fix #2 cont. Possibly change from 0, 1 to -1.0, 1.0? (since discriminator uses sigmoid)
 x_fake = generator.predict(noise)

 discriminator_metric_real = discriminator.train_on_batch(x_real, y_real)
 discriminator_metric_generated = discriminator.train_on_batch(x_fake, y_fake)

 discriminator_metric = 0.5 * np.add(discriminator_metric_real, discriminator_metric_generated)
 #if epoch % 2 == 0:
  #  generator_metric = combined.train_on_batch(noise, y_real)


 generator_metric = combined.train_on_batch(noise, y_real)

 if epoch %20 == 0:
   save_images(cnt, fixed_noise)
   cnt += 1

   print(f"{epoch} epoch, Discriminator accuracy: {100*  discriminator_metric[1]}, Generator accuracy: {100 * generator_metric[1]}")




Model: "sequential_91"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_91 (Dense)            (None, 4096)              413696    
                                                                 
 reshape_45 (Reshape)        (None, 4, 4, 256)         0         
                                                                 
 up_sampling2d_223 (UpSampl  (None, 8, 8, 256)         0         
 ing2D)                                                          
                                                                 
 conv2d_403 (Conv2D)         (None, 8, 8, 256)         590080    
                                                                 
 batch_normalization_313 (B  (None, 8, 8, 256)         1024      
 atchNormalization)                                              
                                                                 
 activation_266 (Activation  (None, 8, 8, 256)       

In [ ]:
!rm -rf output

# New Section